# 3 Pointers Made against shooting.csv

### Import packages

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
from scipy.stats import pearsonr
import itertools

pd.set_option("display.max_columns", None)

### Set working directory

In [2]:
# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

# Change working directory
os.chdir('/Users/tyler/OneDrive/Documents/Python/NBA')

# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA\backend\analysis\3p
Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA


## Exploratory Data Analysis

### Import data

In [3]:
df = pd.read_csv('backend/data/shooting.csv').drop(['Unnamed: 0'], axis=1)

### Basic exploration

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203850 entries, 0 to 203849
Data columns (total 17 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   date      203850 non-null  object 
 1   visitor   203850 non-null  object 
 2   home      203850 non-null  object 
 3   team      203850 non-null  int64  
 4   quarter   203828 non-null  object 
 5   fg        203812 non-null  float64
 6   fga       203812 non-null  float64
 7   fg_perc   203812 non-null  float64
 8   2p        203812 non-null  float64
 9   2pa       203812 non-null  float64
 10  2p_perc   203809 non-null  float64
 11  3p        203812 non-null  float64
 12  3pa       203812 non-null  float64
 13  3p_perc   202349 non-null  float64
 14  efg_perc  203812 non-null  float64
 15  ast       203812 non-null  float64
 16  ast_perc  203789 non-null  float64
dtypes: float64(12), int64(1), object(4)
memory usage: 26.4+ MB


In [5]:
df.head()

,date,visitor,home,team,quarter,fg,fga,fg_perc,2p,2pa,2p_perc,3p,3pa,3p_perc,efg_perc,ast,ast_perc
0,"Tue, Oct 31, 2006",Chicago Bulls,Miami Heat,0,q1,5.0,20.0,0.250,5.0,16.0,0.313,0.0,4.0,0.000,0.250,3.0,0.600
1,"Tue, Oct 31, 2006",Chicago Bulls,Miami Heat,0,q2,15.0,19.0,0.789,12.0,16.0,0.750,3.0,3.0,1.000,0.868,10.0,0.667
2,"Tue, Oct 31, 2006",Chicago Bulls,Miami Heat,0,q3,8.0,21.0,0.381,5.0,16.0,0.313,3.0,5.0,0.600,0.452,4.0,0.500
3,"Tue, Oct 31, 2006",Chicago Bulls,Miami Heat,0,q4,11.0,19.0,0.579,10.0,18.0,0.556,1.0,1.0,1.000,0.605,5.0,0.455
4,"Tue, Oct 31, 2006",Chicago Bulls,Miami Heat,0,total,39.0,79.0,0.494,32.0,66.0,0.485,7.0,13.0,0.538,0.538,22.0,0.564


In [6]:
# Convert 'date' column to Date object
df['date'] = pd.to_datetime(df['date'])


# Conver 'team' column to Team Name
df['team'] = np.where(df['team'], df['home'], df['visitor'])


# Merge in opponents (see team defensive stats)
df = pd.merge(
    df, 
    df, 
    left_on=['date', 'visitor', 'home', 'quarter'], 
    right_on=['date', 'visitor', 'home', 'quarter'],
    suffixes=('', '_opp'),
    how='left')

df = df[df['team'] != df['team_opp']]


df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 203850 entries, 1 to 407698
Data columns (total 30 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   date          203850 non-null  datetime64[ns]
 1   visitor       203850 non-null  object        
 2   home          203850 non-null  object        
 3   team          203850 non-null  object        
 4   quarter       203828 non-null  object        
 5   fg            203812 non-null  float64       
 6   fga           203812 non-null  float64       
 7   fg_perc       203812 non-null  float64       
 8   2p            203812 non-null  float64       
 9   2pa           203812 non-null  float64       
 10  2p_perc       203809 non-null  float64       
 11  3p            203812 non-null  float64       
 12  3pa           203812 non-null  float64       
 13  3p_perc       202349 non-null  float64       
 14  efg_perc      203812 non-null  float64       
 15  ast           203

In [7]:
total_df = df[df['quarter'] == 'total']
q1_df = df[df['quarter'] == 'q1']
q2_df = df[df['quarter'] == 'q2']
q3_df = df[df['quarter'] == 'q3']
q4_df = df[df['quarter'] == 'q4']

In [8]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40218 entries, 9 to 407698
Data columns (total 30 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          40218 non-null  datetime64[ns]
 1   visitor       40218 non-null  object        
 2   home          40218 non-null  object        
 3   team          40218 non-null  object        
 4   quarter       40218 non-null  object        
 5   fg            40202 non-null  float64       
 6   fga           40202 non-null  float64       
 7   fg_perc       40202 non-null  float64       
 8   2p            40202 non-null  float64       
 9   2pa           40202 non-null  float64       
 10  2p_perc       40202 non-null  float64       
 11  3p            40202 non-null  float64       
 12  3pa           40202 non-null  float64       
 13  3p_perc       40202 non-null  float64       
 14  efg_perc      40202 non-null  float64       
 15  ast           40202 non-null  float

# Dataframe of team's last 15 performances

In [9]:
# Totals
total_df = df[df['quarter'] == 'total'].copy()

# Return ten lastest dates team played
def last_15_date(team, date):
    schedule = total_df[total_df['team'] == team].sort_values(by='date').reset_index()
    date_index = schedule[schedule['date'] == date].index[0]
    if date_index - 15 < 0:
        return None, None, None, None, None, None, None, None, None, None, None, None, None, None, None
    else:
        date_1, date_2 = schedule.iloc[date_index - 1]['date'], schedule.iloc[date_index - 2]['date']
        date_3, date_4 = schedule.iloc[date_index - 3]['date'], schedule.iloc[date_index - 4]['date']
        date_5, date_6 = schedule.iloc[date_index - 5]['date'], schedule.iloc[date_index - 6]['date']
        date_7, date_8 = schedule.iloc[date_index - 7]['date'], schedule.iloc[date_index - 8]['date']
        date_9, date_10 = schedule.iloc[date_index - 9]['date'], schedule.iloc[date_index - 10]['date']
        date_11, date_12 = schedule.iloc[date_index - 11]['date'], schedule.iloc[date_index - 12]['date']
        date_13, date_14 = schedule.iloc[date_index - 13]['date'], schedule.iloc[date_index - 14]['date']
        date_15 = schedule.iloc[date_index - 15]['date']
        return date_1, date_2, date_3, date_4, date_5, date_6, date_7, date_8, date_9, date_10, date_11, date_12, date_13, date_14, date_15

total_df['dates'] = total_df.apply(lambda x: last_15_date(x.team, x.date), axis=1)
total_df['date_1'], total_df['date_2'] = total_df['dates'].apply(lambda x: x[0]), total_df['dates'].apply(lambda x: x[1])
total_df['date_3'], total_df['date_4'] = total_df['dates'].apply(lambda x: x[2]), total_df['dates'].apply(lambda x: x[3])
total_df['date_5'], total_df['date_6'] = total_df['dates'].apply(lambda x: x[4]), total_df['dates'].apply(lambda x: x[5])
total_df['date_7'], total_df['date_8'] = total_df['dates'].apply(lambda x: x[6]), total_df['dates'].apply(lambda x: x[7])
total_df['date_9'], total_df['date_10'] = total_df['dates'].apply(lambda x: x[8]), total_df['dates'].apply(lambda x: x[9])
total_df['date_11'], total_df['date_12'] = total_df['dates'].apply(lambda x: x[10]), total_df['dates'].apply(lambda x: x[11])
total_df['date_13'], total_df['date_14'] = total_df['dates'].apply(lambda x: x[12]), total_df['dates'].apply(lambda x: x[13])
total_df['date_15'] = total_df['dates'].apply(lambda x: x[14])

In [10]:
# Define statistics
stats = ['fg', 'fga', '2p', '2pa', '3p', '3pa', 'ast',
         'fg_opp', 'fga_opp', '2p_opp', '2pa_opp', '3p_opp', '3pa_opp', 'ast_opp']

perc_stats = ['fg_perc', '2p_perc', '3p_perc', 'efg_perc', 'ast_perc',
              'fg_perc_opp', '2p_perc_opp', '3p_perc_opp', 'efg_perc_opp', 'ast_perc_opp']

# X and y column names to merge on
x_cols = ['date', 'team'] + stats

last_15_games = total_df.copy()
last_15_games.loc[:, 'target'] = last_15_games.loc[:, '3p']
X = total_df[x_cols].copy()

# Dataframe of target (3pt made by each team) and of variables (last 5 games stats for each team)
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
for date in dates:
    last_15_games = pd.merge(last_15_games, X, left_on=['date' + date, 'team'], right_on=['date', 'team'], how='left', suffixes=('', date))

In [11]:
# Calculate z-score
def z_score(value, mean, std):
    return (value - mean) / std

### Last 15 Performances (Unweighted)

In [12]:
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))] 

last_15 = last_15_games[cols].copy()

# Calculate mean for each stat over a team's last performance
for stat in stats:
    last_15[stat] = 0
    for date in dates:
        last_15[stat] = last_15[stat] + last_15[stat + date]
    
    last_15[stat] = last_15[stat] / len(dates)
    
# Calculate standard deviation for each stat over a team's last 15 game performance
for stat in stats:
    last_15[stat + '_std'] = 0
    for date in dates:
        last_15[stat + '_std'] = last_15[stat + '_std'] + ((last_15[stat + date] - last_15[stat]) ** 2)
    
    last_15[stat + '_std'] = last_15[stat + '_std'] / len(dates)
    last_15[stat + '_std'] = last_15[stat + '_std'] ** .5

# Feature engineer trends (how a team is trending in their last 3 games)
for stat in stats:
    last_15[stat + '_trend'] = 0
    for date in dates[:3]:
        last_15[stat + '_trend'] = last_15[stat + '_trend'] + \
                                    z_score(last_15[stat + date], last_15[stat], last_15[stat + '_std']).fillna(0)
    
    last_15[stat + '_trend'] = last_15[stat + '_trend'] / len(dates[:3])

# Sum stats for opposing teams for each game
last_15 = last_15.groupby(['date', 'visitor', 'home']).sum()

# Standard deviation and trending cols
std_cols = [stat + '_std' for stat in stats]
trend_cols = [stat + '_trend' for stat in stats]

# Keep columns
last_15 = last_15[['target'] + stats + std_cols + trend_cols].dropna(axis=0)

for perc in perc_stats:
    stat = perc.split('_')[0]
    opp = perc.split('_')[-1]
    if opp == 'opp':
        if stat == 'ast':
            last_15[perc] = last_15[stat + '_opp'] / last_15['fg_opp']
        elif stat == 'efg':
            last_15[perc] = (last_15['fg_opp'] + (.5 * last_15['3p_opp'])) / last_15['fga_opp']
        else:
            last_15[perc] = last_15[stat + '_opp'] / last_15[stat + 'a_opp']
    else:
        if stat == 'ast':
            last_15[perc] = last_15[stat] / last_15['fg']
        elif stat == 'efg':
            last_15[perc] = (last_15['fg'] + (.5 * last_15['3p'])) / last_15['fga']
        else:
            last_15[perc] = last_15[stat] / last_15[stat + 'a']

        
last_15 = last_15.dropna(axis=0)
last_15.sample(5)

,,,target,fg,fga,2p,2pa,3p,3pa,ast,fg_opp,fga_opp,2p_opp,2pa_opp,3p_opp,3pa_opp,ast_opp,fg_std,fga_std,2p_std,2pa_std,3p_std,3pa_std,ast_std,fg_opp_std,fga_opp_std,2p_opp_std,2pa_opp_std,3p_opp_std,3pa_opp_std,ast_opp_std,fg_trend,fga_trend,2p_trend,2pa_trend,3p_trend,3pa_trend,ast_trend,fg_opp_trend,fga_opp_trend,2p_opp_trend,2pa_opp_trend,3p_opp_trend,3pa_opp_trend,ast_opp_trend,fg_perc,2p_perc,3p_perc,efg_perc,ast_perc,fg_perc_opp,2p_perc_opp,3p_perc_opp,efg_perc_opp,ast_perc_opp
date,visitor,home,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-03-01,Dallas Mavericks,Brooklyn Nets,16.0,74.200000,165.000000,57.933333,123.600000,16.266667,41.400000,44.666667,74.666667,165.066667,60.266667,126.200000,14.400000,38.866667,44.466667,7.872558,11.072130,7.328872,14.084617,7.089962,12.491653,9.121985,7.960036,12.453651,7.049738,15.594242,6.990003,11.104011,8.073163,-0.971260,-0.364146,-0.222269,0.190079,-0.679454,-0.402056,-0.650763,-1.300582,-1.600126,-1.751253,-0.838860,0.526861,-0.519015,-0.854698,0.449697,0.468716,0.392915,0.498990,0.601977,0.452342,0.477549,0.370497,0.495961,0.595536
2010-02-02,Golden State Warriors,Houston Rockets,13.0,76.933333,171.133333,62.800000,129.200000,14.133333,41.933333,42.000000,77.666667,166.266667,65.533333,132.466667,12.133333,33.800000,39.000000,8.613550,16.376449,8.852578,12.539405,5.018773,10.241969,8.676352,7.851308,15.818170,7.215831,17.175711,5.322330,9.797776,8.395212,0.509386,0.044670,0.649227,0.275814,-0.430722,-0.190376,-0.009717,-0.250980,-0.810625,0.064522,-0.572984,-0.092032,-0.298234,0.052998,0.449552,0.486068,0.337043,0.490845,0.545927,0.467121,0.494716,0.358974,0.503609,0.502146
2014-04-02,Memphis Grizzlies,Minnesota Timberwolves,9.0,78.066667,167.200000,66.266667,133.800000,11.800000,33.400000,46.400000,77.400000,165.733333,61.333333,122.200000,16.066667,43.533333,42.800000,9.022239,11.615467,8.436371,11.245611,3.785557,7.814863,8.030031,8.639697,14.421634,8.421700,16.989200,6.280810,10.926520,8.041016,1.284290,0.493494,1.537665,0.774978,-0.437820,-0.393692,1.422702,0.860106,0.056916,0.755790,-0.436831,0.269730,0.475977,0.453545,0.466906,0.495267,0.353293,0.502193,0.594364,0.467015,0.501909,0.369066,0.515487,0.552972
2008-12-08,Orlando Magic,Los Angeles Clippers,15.0,72.533333,160.666667,58.400000,119.333333,14.133333,41.333333,38.466667,72.933333,164.600000,62.066667,133.666667,10.866667,30.933333,40.400000,6.693873,14.343520,7.461797,14.947885,4.213160,7.002004,5.566524,7.502718,12.135703,7.707193,12.579725,4.729124,8.560747,7.316224,0.174116,-0.478631,0.005428,-0.267725,0.223038,-0.313546,-0.396425,-0.418005,-1.111851,-0.184611,-0.252928,-0.303454,-1.466227,-0.625124,0.451452,0.489385,0.341935,0.495436,0.530331,0.443094,0.464339,0.351293,0.476104,0.553931
2013-05-03,New York Knicks,Boston Celtics,15.0,73.866667,160.266667,56.066667,112.333333,17.800000,47.933333,37.733333,69.066667,155.066667,53.466667,111.133333,15.600000,43.933333,40.133333,11.086509,15.007235,9.672260,16.124200,5.780448,8.957335,9.328774,8.742853,14.668954,9.564156,17.075680,5.654457,8.441978,11.132121,-1.610905,-0.860242,-1.272749,-0.699373,-0.203293,0.042783,-0.866438,-1.017297,-0.073508,-0.941692,-0.487974,0.192042,0.672476,-1.102948,0.460899,0.499110,0.371349,0.516431,0.510830,0.445400,0.481104,0.355083,0.495701,0.581081


## Correlations

In [13]:
corr_df = pd.DataFrame()

# Correlations for last 15 game stats vs 3pt made (unweighted)
for col in last_15:
    corr_p = pearsonr(last_15['target'], last_15[col])
    row = {'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Print correlation
corr_df = corr_df[corr_df['p-value'] < .05].drop(['p-value'], axis=1).sort_values(['corr'], axis=0, ascending=False)
corr_df

,corr,stat
0,1.00,target
6,0.74,3pa
5,0.73,3p
13,0.69,3pa_opp
12,0.68,3p_opp
46,0.52,efg_perc
44,0.52,2p_perc
9,0.50,fga_opp
19,0.49,3p_std
49,0.48,2p_perc_opp


## Save dataframe with significantly correlated stats

In [22]:
stats = corr_df[corr_df['corr'].abs() >= .6]['stat']
df = last_15[stats]

df.to_csv('backend/data/inputs/3p/shooting.csv')